<a id='TOC'></a>

# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, append them into a single dataframe.

# [Resume Here](#ResumeHere)

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    a. [Setting the Location](#SettingLoc)
    
    b. [Importing Necessary Packages](#ImportingPackages)
    
    c. [Preparing for a Save](#PreparingSave)
    

2. [Reading in Data](#ReadingData)

3. [Cleaning Data](#CleaningData)

4. [Appending Data](#AppendingData)

5. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
import numpy as np
import glob,os
from liz_number import lizsort,mindate,smallest,validate
from liz_toes import make_str,label_pattern, replace_pattern,report_pattern

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_config_file(world_readable=True)

# increase print limit
pd.options.display.max_rows = 99999

<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [2]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [3]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [4]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

<a id='ReadingData'></a>

# Reading in Data
[Top](#TOC)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [5]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

Let's inspect these data.  We will begin by looking at the number of columns and rows. In each file. To do this first we define a function, *xlsheets*, which we define below.

In [6]:
def xlsheets(file):
    """The function takes a string, *file*, and returns the number and names of sheets in that file.\
    The function relies on the pandas package."""
    tmp = pd.ExcelFile(file)
    res = {'num_sheets':len(tmp.sheet_names),'names': tmp.sheet_names}
    return res

You can see that when we call *xlsheets* on the first file in our list of files, it returns the total number of sheets and names of each sheet.

In [7]:
xlsheets(rawfiles[0])

{'num_sheets': 4, 'names': ['2000', '2001', '2002', '2003']}

<a id='ResumeHere'></a>

We will apply this to all of the files in raw files to inspect the data at the highest level.

In [44]:
filelayout = pd.Series(rawfiles).apply(xlsheets)
filelayout

0     {'num_sheets': 4, 'names': ['2000', '2001', '2...
1                 {'num_sheets': 1, 'names': ['2004 ']}
2                  {'num_sheets': 1, 'names': ['2015']}
3                  {'num_sheets': 1, 'names': ['2016']}
4                  {'num_sheets': 1, 'names': ['2017']}
5                  {'num_sheets': 1, 'names': ['2005']}
6                  {'num_sheets': 1, 'names': ['2006']}
7        {'num_sheets': 2, 'names': ['Sheet1', '2007']}
8                  {'num_sheets': 1, 'names': ['2008']}
9                  {'num_sheets': 1, 'names': ['2009']}
10               {'num_sheets': 1, 'names': ['Sheet1']}
11               {'num_sheets': 1, 'names': ['Sheet1']}
12                 {'num_sheets': 1, 'names': ['data']}
13         {'num_sheets': 1, 'names': ['CC 2013 data']}
14                 {'num_sheets': 1, 'names': ['2014']}
dtype: object

In [80]:
filelayout = pd.DataFrame([list(file.values()) for file in filelayout])
filelayout.columns = ['num_sheets','sheetnames']
filelayout

,num_sheets,sheetnames
0,4,"[2000, 2001, 2002, 2003]"
1,1,[2004 ]
2,1,[2015]
3,1,[2016]
4,1,[2017]
5,1,[2005]
6,1,[2006]
7,2,"[Sheet1, 2007]"
8,1,[2008]
9,1,[2009]


In [90]:
filelayout.sheetnames.str.split(',')

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
Name: sheetnames, dtype: float64

We will need to treat the 1st and 8th files in the source data differently when we read in the data.  Let's group the file names accordingly.

In [9]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [10]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final.xls', 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [11]:
rawfiles_ss

['xCC2011x.xls',
 'CC 2016 - captures.xls',
 'xCC2012x.xls',
 'xCC2010x.xlsx',
 'xCC2013x.xls',
 'xCC2008x.xls',
 'CC 2015 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2006x.xls',
 'xCC2014x.xlsx',
 'CC 2004.xlsx',
 'xCC2005x.xls',
 'xCC2009x.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will define another function, *xlcolshape* to make this easier.

In [12]:
def xlcolshape(file):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    return pd.read_excel(file).shape

When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns').

In [13]:
xlcolshape(rawfiles_ss[0])

(64, 19)

We will apply this funtion to the list of single-sheet files for our inspection.

In [14]:
pd.Series(rawfiles_ss).apply(xlcolshape)

0      (64, 19)
1     (103, 21)
2      (85, 19)
3      (99, 41)
4     (106, 19)
5     (134, 20)
6     (241, 19)
7     (798, 21)
8     (163, 16)
9      (97, 19)
10    (479, 16)
11    (202, 16)
12    (162, 16)
dtype: object

<a id='CleaningData'></a>

# Cleaning Data
[Top](#TOC)

<a id='HandlingColumns'></a>

## Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [build a list of unique column names](#FindUniqueCol)
2. [eliminate unnecessary columns](#DropCol)
3. [combine synonyms ](#CombineCol)

<a id='FindUniqueCol'></a>

### Finding Unique Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

We'll define a function to extract column names and convert them to a set we'll use that function to allow us to only add unique names to a list of column names.

In [15]:
def xluniquecol(file,sheet=0):
    """xluniquecol takes a string filename of an excel file and extracts the column names as a set."""
    return list(pd.read_excel(file,sheet_name=sheet).columns)


Here is an example of how xluniquecol works.

In [16]:
xluniquecol(rawfiles_ss[0])

['Species',
 'Toes',
 'Date',
 'Sex',
 'SVL',
 'TL',
 'RTL',
 'Mass',
 'Paint Mark',
 'Location',
 'Meters',
 'New/Recap',
 'Painted',
 'Misc.',
 'Vial',
 'Time',
 'Unnamed: 16',
 'Spotted',
 'Mark']

Now we will create an empty set, *uniquecols*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols*.

In [17]:
tmp = pd.Series(rawfiles_ss).apply(xluniquecol)
uniquecols = list()
for u in tmp:
    uniquecols = uniquecols+u
uniquecols = list(set(uniquecols))
uniquecols

['TIME',
 1,
 'painted or not',
 'paint mark',
 'misc/notes',
 'Toe 18',
 '2015 or earlier',
 'TL (mm)',
 'Misc.',
 'RTL',
 'Mark',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'Toe 9',
 'TOES',
 'Toes',
 'Unnamed: 16',
 'Toe 15',
 'Years Alive (known)',
 'painted',
 'mass (g)',
 'Painted',
 'misc',
 'Unnamed: 19',
 'Toe 11',
 'Toe 4',
 ' painted or not',
 'meters',
 'Collectors',
 'date',
 'Date',
 'Meters',
 'SVL',
 'Toe 1',
 'Toe 3',
 'Toe 2',
 'Mass',
 'RTL (mm)',
 'Toe 5',
 'Toe 12',
 'Toe 13',
 '1st Capture (year)',
 'Toe 14',
 'Sex',
 'mass',
 'SVL (mm)',
 'Vial',
 'NEW/recap',
 'VIAL',
 'sex',
 'Location',
 'Time',
 'TL',
 'Spotted',
 'Toe 19',
 'Paint Mark',
 'Toe 16',
 'Year',
 'Toe 7',
 'New/Recap',
 'species',
 'Species',
 'Toe 6',
 'Toe 10',
 'Marked',
 'Toe 17',
 'location',
 'Toe 8',
 'Toe 20',
 'Unnamed: 17']

<a id='DropCol'></a>

### Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [18]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes']

In [19]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols).str.lower())

{'autotomized', 'new.recap', 'paint.mark', 'sighting'}

In [20]:
set(pd.Series(uniquecols).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'year',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

### Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

Let's define a function to identify potential synonyms. One approach is to take a column name that we want to keep and search through a list of names to identify any that contain enough of that name to be considered potentially synonymous.  Let's try to return these potential matches as values in a dictionary and return the desired name as the key for those values.

In [21]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key."""
    
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

In [22]:
coldict = {}

In [23]:
colmatchtodict('toes',pd.Series(uniquecols),coldict, key = 'toes')

{'toes': ['TOES', 'Toes']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [24]:
coldict = {}

In [25]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols),dictsource=coldict))
coldict

{'species': ['species', 'Species'],
 'date': ['date', 'Date'],
 'sex': ['Sex', 'sex'],
 'svl': ['SVL', 'SVL (mm)'],
 'tl': ['TL (mm)', 'RTL', 'RTL (mm)', 'TL'],
 'rtl': ['RTL', 'RTL (mm)'],
 'mass': ['mass (g)', 'Mass', 'mass'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['Location', 'location'],
 'meters': ['meters', 'Meters'],
 'new.recap': ['NEW/recap', 'New/Recap'],
 'painted': ['painted or not', 'painted', 'Painted', ' painted or not'],
 'misc': ['misc/notes', 'Misc.', 'misc'],
 'vial': ['Vial', 'VIAL'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['TOES', 'Toes']}

We will manually adjust the values for 'tl'.

In [26]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We need a function that will take a value in *uniquecols*

In [27]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res

Here is are a few examples of how *findsyn* works.

In [28]:
findsyn('RTi',coldict,verbose=False)

In [29]:
findsyn('RTi',coldict,verbose=True)

No value matching "RTi" was found in the dictionary.


In [30]:
findsyn('RTL',coldict,verbose=True)

'rtl'

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [31]:
uniquecolsdf = pd.DataFrame({'uniquecols':uniquecols})
uniquecolsdf['preferredcol'] = uniquecolsdf.uniquecols.apply(lambda x: findsyn(x,coldict,False))
uniquecolsdf

,uniquecols,preferredcol
0,TIME,None
1,1,None
2,painted or not,painted
3,paint mark,paint.mark
4,misc/notes,misc
5,Toe 18,None
6,2015 or earlier,None
7,TL (mm),tl
8,Misc.,misc
9,RTL,rtl


Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [32]:
uniquecolsdf.index = uniquecolsdf.uniquecols
uniquecolsdict = pd.Series(uniquecolsdf.preferredcol).to_dict()
uniquecolsdict

{'TIME': None,
 1: None,
 'painted or not': 'painted',
 'paint mark': 'paint.mark',
 'misc/notes': 'misc',
 'Toe 18': None,
 '2015 or earlier': None,
 'TL (mm)': 'tl',
 'Misc.': 'misc',
 'RTL': 'rtl',
 'Mark': None,
 'Tail condition (1=intact; 2=autotomized; 3=regrown)': 'autotomized',
 'Toe 9': None,
 'TOES': 'toes',
 'Toes': 'toes',
 'Unnamed: 16': None,
 'Toe 15': None,
 'Years Alive (known)': None,
 'painted': 'painted',
 'mass (g)': 'mass',
 'Painted': 'painted',
 'misc': 'misc',
 'Unnamed: 19': None,
 'Toe 11': None,
 'Toe 4': None,
 ' painted or not': 'painted',
 'meters': 'meters',
 'Collectors': None,
 'date': 'date',
 'Date': 'date',
 'Meters': 'meters',
 'SVL': 'svl',
 'Toe 1': None,
 'Toe 3': None,
 'Toe 2': None,
 'Mass': 'mass',
 'RTL (mm)': 'rtl',
 'Toe 5': None,
 'Toe 12': None,
 'Toe 13': None,
 '1st Capture (year)': None,
 'Toe 14': None,
 'Sex': 'sex',
 'mass': 'mass',
 'SVL (mm)': 'svl',
 'Vial': 'vial',
 'NEW/recap': 'new.recap',
 'VIAL': 'vial',
 'sex': 'sex',
 'L

We'll use the dict, *uniquecolsdict* to rename the synonymous columns in our file....once we read them in,
that is.

<a id='AppendingData'></a>

# Appending Data
[Top](#TOC)

In [38]:
def readnclean(x,dictionary,dtype=None):
    """
    This function reads an excel file, renames columns deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    tmp = pd.read_excel(x,dtype=dtype)
    tmp.columns = pd.Series(tmp.columns).map(lambda x:dictionary[x])
    dropidx =[None==col for col in list(tmp.columns)]
    tmp=tmp.drop(columns=tmp.columns[dropidx])
    
    return tmp

Here is an example of how *readnclean* works.

In [39]:
readnclean(rawfiles_ss[0],uniquecolsdict,dtype=str).head()

,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sj,4 - 6 - 12,2011-06-20 00:00:00,M,68,101,0,9.2,g20b,15m up CCC,255,new,yes,recently shed,08 - 11
1,sj,4 - 6 - 13,2011-06-28 00:00:00,M,77,106,0,13.8,g40b,big juniper right 8m ^ juniper Xing,203,new,yes,nan,21 - 11
2,Uo,6 - 15,2011-06-20 00:00:00,M,49,83,0,4,g.t,opp oak R,418,new,yes,nan,09 - 11
3,uo,6 - 16,2011-06-28 00:00:00,F,49,72,0,4.2,g.b,CC/CCC,240,new,yes,Orange throat; gravid,22 - 11
4,Uo,6 - 17,2011-07-02 00:00:00,F,48,69,0,4,g.c,H4b,200,new,yes,nan,27-11


We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [40]:
df = pd.DataFrame(columns=coldict.keys())
df

,species,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial,autotomized,sighting,toes


Now we will read in all of the raw_ss files clean the column names and concatenate them into one large df.

In [43]:
for file in rawfiles_ss:
    df = pd.concat([df,readnclean(file,uniquecolsdict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

5530
5633
5718
5817
5923
6057
6298
7096
7259
7356
7835
8037
8199


Final df has 17 columns and 8199 rows.


,autotomized,date,location,mass,meters,misc,new.recap,paint.mark,painted,rtl,sex,sighting,species,svl,tl,toes,vial
0.0,NaN,2011-06-20 00:00:00,15m up CCC,9.2,255,recently shed,new,g20b,yes,0,M,NaN,sj,68,101,4 - 6 - 12,08 - 11
1.0,NaN,2011-06-28 00:00:00,big juniper right 8m ^ juniper Xing,13.8,203,NaN,new,g40b,yes,0,M,NaN,sj,77,106,4 - 6 - 13,21 - 11
2.0,NaN,2011-06-20 00:00:00,opp oak R,4,418,NaN,new,g.t,yes,0,M,NaN,Uo,49,83,6 - 15,09 - 11
3.0,NaN,2011-06-28 00:00:00,CC/CCC,4.2,240,Orange throat; gravid,new,g.b,yes,0,F,NaN,uo,49,72,6 - 16,22 - 11
4.0,NaN,2011-07-02 00:00:00,H4b,4,200,NaN,new,g.c,yes,0,F,NaN,Uo,48,69,6 - 17,27-11


For the *rawdata_ms* files we will apply a similar logic

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)